Notebook to download first image in each recipe and save to parquet file

In [3]:
import torch
import torchvision as tv
import torchvision.transforms.v2 as t
from torchvision import utils
import polars as pl
import requests
from PIL import Image
from io import BytesIO
import os
import warnings

In [4]:
df=pl.scan_parquet(
    path+'recipes.parquet'
).select(
    'RecipeId',
    pl.col('Images').list.get(0),
    pl.col('AggregatedRating').fill_null(1)>=4
).filter(
    pl.col('Images').str.slice(-4).str.to_lowercase().is_in(['.jpg','jpeg','.png']),
)

In [5]:
tr = t.Compose([
    t.ToImage(),
    t.Resize([256], interpolation=t.InterpolationMode.BICUBIC, antialias=True),
    t.ToPILImage()
])

In [29]:
def f(url):
    try:
        im = Image.open(BytesIO(requests.get(url).content))
    except:
        return BytesIO(b'~~ERROR Exception Thrown ERROR~~').getvalue()
    if im.mode != 'RGB':
        im = im.convert('RGB')
    b = BytesIO()
    tr(im).save(b, format='JPEG')
    return b.getvalue()

In [21]:
%%time
df.with_columns(
    pl.col('Images').map_elements(f, return_dtype=pl.Binary)
).sink_parquet(
    'images_df.parquet'
)

CPU times: user 437 ms, sys: 92.9 ms, total: 530 ms
Wall time: 415 ms
